#1 Introducción

El siguiente cuaderno realiza la búsqueda secuencial [3] utilizando el GPGPU de un número ingresado por teclado en un vector de números flotantes y se informa si se lo encontro o no. El algoritmo de este cuadernos se basa en, valga la redundancia, el ya conocido algoritmo de búsqueda secuencial o lineal [3]

El objetivo es comparar los tiempos de ejecución de ambas versiones del algoritmo de busqueda, utilizando el lenguaje python [2] en la plataforma colab [1] y CUDA [4,5].

#2 Armado del ambiente

Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 10.2MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621228 sha256=4d0f4bb0ec2ed9e432f398c48df718fad3d56d42906ebd5c9504f806e05f8306
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=519d078206380172d6f2d4af96bf2e63e56959c2fb46acf7de14eeec19ff9abb
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

In [ ]:
#@title Parametros { vertical-output: true, display-mode: "both" }
numeroElementos =  6#@param {type:"number"}

try:
  from datetime import datetime
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule

  from  datetime import datetime
  import numpy

  tiempoTotal = datetime.now()

  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  # CPU - Defino la memoria del vector en cpu.
  vectorCpu = numpy.random.randn(numeroElementos)
  vectorCpu = vectorCpu.astype(numpy.float32())
  print("vector con numeros:")
  print(vectorCpu)

  # CPU - reservo la memoria GPU.
  vectorGpu = cuda.mem_alloc(vectorCpu.nbytes)

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(vectorGpu,vectorCpu)

  # CPU - ingreso el numero a buscar
  print("Ingrese un numero a buscar:")
  numeroBuscado = input()

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void buscarNumero(int n,float numeroBuscado ,float *pvector1)
  {
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      
      

      if(idx<n)
      {
        if(pvector1[idx] == numeroBuscado)
        {
            pvector1[0] = 1;
        }
      }
  }

  """)

  kernel = module.get_function("buscarNumero")
  tiempo_gpu = datetime.now()

  dim_hilo = 256
  dim_bloque = numpy.int( (numeroElementos+dim_hilo-1) / dim_hilo )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

  kernel( numpy.int32(numeroElementos),numpy.float32(numeroBuscado), vectorGpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

  tiempo_gpu = datetime.now() - tiempo_gpu
  cuda.memcpy_dtoh(vectorCpu,vectorGpu)

  print("vector con numeros final:")
  print(vectorCpu)
  
  # CPU - Informo el resultado.
  if vectorCpu[0]== 1:
    print("El numero ha sido encontrado")
  else:
      print("El numero no ha sido encontrado")


  tiempoTotal = datetime.now() - tiempoTotal
  print("")
  print("Tiempo GPU: ",tiempo_en_ms(tiempo_gpu),"[ms]")
  print("Tiempo CPU: ",tiempo_en_ms(tiempoTotal),"[ms]")


except ModuleNotFoundError :
      print("No se compilo el código del armado del ambiente")
      print("Compile el armado del ambiente y vuelva a intentarlo")
except ValueError :
  print("se ingreso un número de elementos menor a 0")
  print("ingrese un número de elementos mayor a 0")
except pycuda.driver.LogicError:
  print("Ha ingresado un número de elementos igual a 0 y CUDA no puede reservar un vector de tamaño 0")
  print("ingrese un número de elementos mayor a 0")
      

vector con numeros:
[ 0.739824    0.21022892 -0.6425146   0.06134877 -1.1046423  -2.047388  ]
Ingrese un numero a buscar:
0.739824
Thread x:  256 , Bloque x: 1
vector con numeros final:
[ 1.          0.21022892 -0.6425146   0.06134877 -1.1046423  -2.047388  ]
El numero ha sido encontrado

Tiempo GPU:  0.524 [ms]
Tiempo CPU:  4358.174 [ms]


#4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( numeroElementos ) | Inicializa el vectorCpu
CPU      |  print()               | informo el vector de números en el que voy a buscar 
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  print()               | informo que se debe ingresar un numero a buscar
CPU      |  input()               | ingreso el numeroBuscado por teclado
CPU      |  SourceModule()        | Define el código del kernel donde se realiza la busqueda de un numero, si se encuentra, en la  posicion cero copio un 1 para usarlo como bandera
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_hilo/dim_bloque   | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  if                    | si la posicion 0 del vectorCpu es igual a 1 significa que lo encontre, de lo contrario no lo encontre
CPU      |  print()               | Informo los resultados.


#5 Conclusiones

En este ejercicio que se basa en el algortimo de una búsqueda secuencial utilizando paralelismo (CPU-GPU), donde se busca un número ingresado por teclado dentro de un vector de números flotantes y se informa si se lo encontro o no, los tiempos de ejecución resultaron más eficientes utilizando solamente la CPU  de manera secuencial porque al ser un algoritmo donde se va comparando todo el tiempo el numero buscado con cada posición del vector tanto en la version secuencial(CPU) como en la versión con paralelismo (CPU-GPU), el costo termina siendo mayor en la version que se utiliza paralelismo con la GPU  por las capas intermedias por las que deben pasar los datos desde que se envian desde la CPU a la GPU hasta que vuelven de la GPU a la CPU,le suman un tiempo mayor al propio dado por la busqueda.

Para poder sacar estas conclusiones tome 20 muestras del algoritmo en la version secuencial (CPU) y la versión con paralelismo (CPU-GPU) con un número de elementos igual a 100 y obtuve los siguientes promedios:

Promedio de tiempo total ejercicio 1 secuencial: 3.239,6 ms

Promedio de tiempo de Bucle ejercicio 1  secuencial: 0,2534 ms 

Promedio de tiempo CPU ejercicio 1 paralelismo: 4.665,4 ms

Promedio de tiempo GPU ejercicio 1 paralelismo: 506,9 ms

Como vemos en los resultados obtenidos, el tiempo total(CPU) en la versión secuencial es menor al tiempo equivalente que es el tiempo de CPU, en la versión con paralelismo.

Por este motivo puedo decir que en los ejercicios donde exista la misma cantidad de comparaciones de datos tanto en la versíon secuencial como en la de paralelismo el tiempo va a ser mayor en la de paralelismo por lo antes mensionado sobre las capas por las que pasan los datos hasta llegar a la GPU y y volver a la CPU.

Para continuar este ejercicio tanto en la versión secuencial como en la versión con paralelismo se podría hacer que una vez que se encuentre el número buscado, en el caso de la versión secuencial, salga del bucle inmediatamente y en la versión con paralelismo los hilos dejen de comparar.


#6 Bibliografia

[1] MARKDOWN SYNTAX Colab:[PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [pagina colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[3] Función de busqueda secuencial: [referencia](https://es.wikipedia.org/wiki/Algoritmo_de_b%C3%BAsqueda)

[4] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[5] Repositorio de PyCUDA: [WEB](https://pypi.org/project/pycuda/)